# 1.NSCLC Sex-stratified SC Analysis

## Data selection, pseudobulk, create input for DESEQ2

Using pertpy as workaround for sccoda installation because of issues with conda environment and scanpy

In [ ]:
import scanpy as sc

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pertpy as pt

In [ ]:
from matplotlib import cm, rcParams

In [ ]:
#import warnings
#import numpy as np
#import pandas as pd 
#import scanpy as sc
#import matplotlib.pyplot as plt
#import matplotlib
#import seaborn as sns

In [ ]:
#import importlib
#import warnings
#warnings.filterwarnings("ignore")
#import pandas as pd
#import pickle as pkl
#import matplotlib.pyplot as plt

In [ ]:
#import seaborn as sns
#import matplotlib.pyplot as plt

In [ ]:
#from statannotations.Annotator import Annotator

In [ ]:
#from scipy.stats import wilcoxon

In [ ]:
path = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/pseudobulk/"
resDir = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/figures/"
input_path = f"{path}/paired_adata_clean.h5ad"

In [ ]:
adata = sc.read_h5ad(input_path) 

## SCCODA

In [ ]:
adata.obs

In [ ]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_type",
    sample_identifier="sample",
    covariate_obs=["origin","dataset","disease","sex","donor_id"],
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
# Select control and salmonella data
sccoda_data.mod["coda_tumor_normal"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["origin"].isin(["normal_adjacent", "tumor_primary"])
].copy()
print(sccoda_data["coda_tumor_normal"])

In [ ]:
sccoda_data

In [ ]:
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)

In [ ]:
plot_df

In [ ]:
significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()


for cell_type in cell_types:
    male_fractions = plot_df[(plot_df['cell_type'] == cell_type) & (plot_df['sex'] == 'male')]['fraction'].dropna()
    female_fractions = plot_df[(plot_df['cell_type'] == cell_type) & (plot_df['sex'] == 'female')]['fraction'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)


In [ ]:
sccoda_data["coda_tumor_normal"].obs["origin_sex"] = sccoda_data["coda_tumor_normal"].obs["origin"].astype(str) + "_" + sccoda_data["coda_tumor_normal"].obs["sex"].astype(str)

In [ ]:
sccoda_data["coda_tumor_normal"]

In [ ]:
plot_df

In [ ]:
data =sccoda_data["coda_tumor_normal"]
feature_name = "origin_sex"
y_scale= "relative"
plot_facets= False
add_dots= False
args_boxplot = {}
args_swarmplot = {}
figsize= None
dpi = 100
cmap= "Blues"
plot_legend = True
level_order= None

# y scale transformations
if y_scale == "relative":
    sample_sums = np.sum(data.X, axis=1, keepdims=True)
    X = data.X/sample_sums
    value_name = "Proportion"
# add pseudocount 1 if using log scale (needs to be improved)
elif y_scale == "log":
    X = np.log(data.X + 1)
    value_name = "log(count)"
elif y_scale == "count":
    X = data.X
    value_name = "count"
else:
    raise ValueError("Invalid y_scale transformation")

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
    merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)

plot_df['origin'] = plot_df['origin_sex'].str.split('_', n=2).str[0]  
plot_df['sex'] = plot_df['origin_sex'].str.split('_', n=2).str[2] 

# Step 3: Perform wilcoxonTest for significance and display p-values
significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()

plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

if level_order:
            args_boxplot["hue_order"] = level_order
            args_swarmplot["hue_order"] = level_order

    
fig, ax = plt.subplots(figsize=(12, 6), dpi=dpi)
feature_name = "sex"
sns.boxplot(x="Cell type", y=value_name, hue=feature_name, data=plot_df, fliersize=1,
            palette=cmap, ax=ax, **args_boxplot)

if add_dots:
    sns.swarmplot(
        x="Cell type",
        y=value_name,
        data=plot_df,
        hue=feature_name,
        ax=ax,
        dodge=True,
        color="black",
        **args_swarmplot
    )

significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()


for cell_type in cell_types:
    male_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['sex'] == 'male')]['Proportion'].dropna()
    female_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['sex'] == 'female')]['Proportion'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    #if p_value < significance_level:
x_position = list(cell_types).index(cell_type)
    y_max = plot_df[plot_df['Cell type'] == cell_type]['Proportion'].max()
    plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)
        
cell_types = pd.unique(plot_df["Cell type"])
ax.set_xticklabels(cell_types, rotation=90)

if plot_legend:
    handles, labels = ax.get_legend_handles_labels()
    handout = []
    labelout = []
    for h, l in zip(handles, labels):
        if l not in labelout:
            labelout.append(l)
            handout.append(h)
    ax.legend(handout, labelout, loc='upper left', bbox_to_anchor=(1, 1), ncol=1, title=feature_name)
    
# Customize the plot
plt.xlabel('')
plt.ylabel('Cell proportion sccoda')
plt.title('Cell Type Fractions. Sex and sample. Tumor & Normal. Wilcoxon test')
plt.tight_layout()


In [ ]:
data =sccoda_data["coda_tumor_normal"][sccoda_data["coda_tumor_normal"].obs["sex"]=="female"]
feature_name = "origin_sex"
y_scale= "relative"
plot_facets= False
add_dots= False
args_boxplot = {}
args_swarmplot = {}
figsize= None
dpi = 100
cmap= "Blues"
plot_legend = True
level_order= None

# y scale transformations
if y_scale == "relative":
    sample_sums = np.sum(data.X, axis=1, keepdims=True)
    X = data.X/sample_sums
    value_name = "Proportion"
# add pseudocount 1 if using log scale (needs to be improved)
elif y_scale == "log":
    X = np.log(data.X + 1)
    value_name = "log(count)"
elif y_scale == "count":
    X = data.X
    value_name = "count"
else:
    raise ValueError("Invalid y_scale transformation")

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
    merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)

plot_df['origin'] = plot_df['origin_sex'].str.split('_', n=2).str[0]  
plot_df['sex'] = plot_df['origin_sex'].str.split('_', n=2).str[2] 

# Step 3: Perform wilcoxonTest for significance and display p-values
significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()

plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

if level_order:
            args_boxplot["hue_order"] = level_order
            args_swarmplot["hue_order"] = level_order

    
fig, ax = plt.subplots(figsize=(12, 6), dpi=dpi)
feature_name = "origin"
sns.boxplot(x="Cell type", y=value_name, hue=feature_name, data=plot_df, fliersize=1,
            palette=cmap, ax=ax, **args_boxplot)

if add_dots:
    sns.swarmplot(
        x="Cell type",
        y=value_name,
        data=plot_df,
        hue=feature_name,
        ax=ax,
        dodge=True,
        color="black",
        **args_swarmplot
    )

significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()


for cell_type in cell_types:
    male_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['origin'] == 'tumor')]['Proportion'].dropna()
    female_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['origin'] == 'normal')]['Proportion'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = plot_df[plot_df['Cell type'] == cell_type]['Proportion'].max()
        plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)
        
cell_types = pd.unique(plot_df["Cell type"])
ax.set_xticklabels(cell_types, rotation=90)

if plot_legend:
    handles, labels = ax.get_legend_handles_labels()
    handout = []
    labelout = []
    for h, l in zip(handles, labels):
        if l not in labelout:
            labelout.append(l)
            handout.append(h)
    ax.legend(handout, labelout, loc='upper left', bbox_to_anchor=(1, 1), ncol=1, title=feature_name)
    
# Customize the plot
plt.xlabel('')
plt.ylabel('Cell proportion sccoda')
plt.title('Cell Type Fractions. Female. Wilcoxon test')
plt.tight_layout()


In [ ]:
data =sccoda_data["coda_tumor_normal"][sccoda_data["coda_tumor_normal"].obs["sex"]=="male"]
feature_name = "origin_sex"
y_scale= "relative"
plot_facets= False
add_dots= False
args_boxplot = {}
args_swarmplot = {}
figsize= None
dpi = 100
cmap= "Blues"
plot_legend = True
level_order= None

# y scale transformations
if y_scale == "relative":
    sample_sums = np.sum(data.X, axis=1, keepdims=True)
    X = data.X/sample_sums
    value_name = "Proportion"
# add pseudocount 1 if using log scale (needs to be improved)
elif y_scale == "log":
    X = np.log(data.X + 1)
    value_name = "log(count)"
elif y_scale == "count":
    X = data.X
    value_name = "count"
else:
    raise ValueError("Invalid y_scale transformation")

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
    merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)

plot_df['origin'] = plot_df['origin_sex'].str.split('_', n=2).str[0]  
plot_df['sex'] = plot_df['origin_sex'].str.split('_', n=2).str[2] 

# Step 3: Perform wilcoxonTest for significance and display p-values
significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()

plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

if level_order:
            args_boxplot["hue_order"] = level_order
            args_swarmplot["hue_order"] = level_order

    
fig, ax = plt.subplots(figsize=(12, 6), dpi=dpi)
feature_name = "origin"
sns.boxplot(x="Cell type", y=value_name, hue=feature_name, data=plot_df, fliersize=1,
            palette=cmap, ax=ax, **args_boxplot)

if add_dots:
    sns.swarmplot(
        x="Cell type",
        y=value_name,
        data=plot_df,
        hue=feature_name,
        ax=ax,
        dodge=True,
        color="black",
        **args_swarmplot
    )

significance_level = 0.05  

cell_types = plot_df['Cell type'].unique()


for cell_type in cell_types:
    male_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['origin'] == 'tumor')]['Proportion'].dropna()
    female_fractions = plot_df[(plot_df['Cell type'] == cell_type) & (plot_df['origin'] == 'normal')]['Proportion'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = plot_df[plot_df['Cell type'] == cell_type]['Proportion'].max()
        plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)
        
cell_types = pd.unique(plot_df["Cell type"])
ax.set_xticklabels(cell_types, rotation=90)

if plot_legend:
    handles, labels = ax.get_legend_handles_labels()
    handout = []
    labelout = []
    for h, l in zip(handles, labels):
        if l not in labelout:
            labelout.append(l)
            handout.append(h)
    ax.legend(handout, labelout, loc='upper left', bbox_to_anchor=(1, 1), ncol=1, title=feature_name)
    
# Customize the plot
plt.xlabel('')
plt.ylabel('Cell proportion sccoda')
plt.title('Cell Type Fractions. Male. Wilcoxon test')
plt.tight_layout()


In [ ]:
import pandas as pd

# Step 1: Count rows per donor_id in each DataFrame
count_tumor_data = tumor_data.groupby('donor_id').size().reset_index(name='count_A')
count_normal_data = normal_data.groupby('donor_id').size().reset_index(name='count_B')

# Step 2: Merge the counts on donor_id
merged_counts = pd.merge(count_tumor_data, count_normal_data, on='donor_id', how='outer')

# Step 3: Identify donor_ids where counts are different
donors_to_remove = merged_counts[merged_counts['count_A'] != merged_counts['count_B']]['donor_id']

# Step 4: Remove donor_ids with mismatched counts from both A and B
tumor_data_cleaned = tumor_data[~tumor_data['donor_id'].isin(donors_to_remove)]
normal_data_cleaned = normal_data[~normal_data['donor_id'].isin(donors_to_remove)]

In [ ]:
#sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="coda_tumor_normal", feature_name="origin")
#plt.show()

In [ ]:
data =sccoda_data["coda_tumor_normal"]
feature_name = "origin"
figsize=(5, 6)
dpi = 100
cmap = cm.tab20
plot_legend = True
level_order = None



# cell type names
type_names = data.var.index

# option to plot one stacked barplot per sample
if feature_name == "samples":
    if level_order:
        assert set(level_order) == set(data.obs.index), "level order is inconsistent with levels"
        data = data[level_order]
    g = stackbar(
        data.X,
        type_names=data.var.index,
        title="samples",
        level_names=data.obs.index,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
else:
    # Order levels
    if level_order:
        assert set(level_order) == set(data.obs[feature_name]), "level order is inconsistent with levels"
        levels = level_order
    elif hasattr(data.obs[feature_name], 'cat'):
        levels = data.obs[feature_name].cat.categories.to_list()
    else:
        levels = pd.unique(data.obs[feature_name])
    n_levels = len(levels)
    feature_totals = np.zeros([n_levels, data.X.shape[1]])

    for level in range(n_levels):
        l_indices = np.where(data.obs[feature_name] == levels[level])
        feature_totals[level] = np.sum(data.X[l_indices], axis=0)

    g = stackbar(
        feature_totals,
        type_names=type_names,
        title=feature_name,
        level_names=levels,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
    g.spines['top'].set_visible(False)
    g.spines['right'].set_visible(False)


    # Add labels and title
    plt.xlabel('')
    plt.ylabel('Proportion of Cell Types')
    plt.title('')
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

In [ ]:
data =sccoda_data["coda_tumor_normal"]
feature_name = "sex"
figsize=(5, 6)
dpi = 100
cmap = cm.tab20
plot_legend = True
level_order = None



# cell type names
type_names = data.var.index

# option to plot one stacked barplot per sample
if feature_name == "samples":
    if level_order:
        assert set(level_order) == set(data.obs.index), "level order is inconsistent with levels"
        data = data[level_order]
    g = stackbar(
        data.X,
        type_names=data.var.index,
        title="samples",
        level_names=data.obs.index,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
else:
    # Order levels
    if level_order:
        assert set(level_order) == set(data.obs[feature_name]), "level order is inconsistent with levels"
        levels = level_order
    elif hasattr(data.obs[feature_name], 'cat'):
        levels = data.obs[feature_name].cat.categories.to_list()
    else:
        levels = pd.unique(data.obs[feature_name])
    n_levels = len(levels)
    feature_totals = np.zeros([n_levels, data.X.shape[1]])

    for level in range(n_levels):
        l_indices = np.where(data.obs[feature_name] == levels[level])
        feature_totals[level] = np.sum(data.X[l_indices], axis=0)

    g = stackbar(
        feature_totals,
        type_names=type_names,
        title=feature_name,
        level_names=levels,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
       # Add labels and title
    plt.xlabel('')
    plt.ylabel('Proportion of Cell Types. T&N')
    plt.title('')
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()


In [ ]:
data =sccoda_data["coda_tumor_normal"][sccoda_data["coda_tumor_normal"].obs["origin"]=="tumor_primary"]
feature_name = "sex"
figsize=(5, 6)
dpi = 100
cmap = cm.tab20
plot_legend = True
level_order = None



# cell type names
type_names = data.var.index

# option to plot one stacked barplot per sample
if feature_name == "samples":
    if level_order:
        assert set(level_order) == set(data.obs.index), "level order is inconsistent with levels"
        data = data[level_order]
    g = stackbar(
        data.X,
        type_names=data.var.index,
        title="samples",
        level_names=data.obs.index,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
else:
    # Order levels
    if level_order:
        assert set(level_order) == set(data.obs[feature_name]), "level order is inconsistent with levels"
        levels = level_order
    elif hasattr(data.obs[feature_name], 'cat'):
        levels = data.obs[feature_name].cat.categories.to_list()
    else:
        levels = pd.unique(data.obs[feature_name])
    n_levels = len(levels)
    feature_totals = np.zeros([n_levels, data.X.shape[1]])

    for level in range(n_levels):
        l_indices = np.where(data.obs[feature_name] == levels[level])
        feature_totals[level] = np.sum(data.X[l_indices], axis=0)

    g = stackbar(
        feature_totals,
        type_names=type_names,
        title=feature_name,
        level_names=levels,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
       # Add labels and title
    plt.xlabel('')
    plt.ylabel('Proportion of Cell Types. Tumor')
    plt.title('')
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()


In [ ]:
data =sccoda_data["coda_tumor_normal"][sccoda_data["coda_tumor_normal"].obs["origin"]=="normal_adjacent"]
feature_name = "sex"
figsize=(5, 6)
dpi = 100
cmap = cm.tab20
plot_legend = True
level_order = None



# cell type names
type_names = data.var.index

# option to plot one stacked barplot per sample
if feature_name == "samples":
    if level_order:
        assert set(level_order) == set(data.obs.index), "level order is inconsistent with levels"
        data = data[level_order]
    g = stackbar(
        data.X,
        type_names=data.var.index,
        title="samples",
        level_names=data.obs.index,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
else:
    # Order levels
    if level_order:
        assert set(level_order) == set(data.obs[feature_name]), "level order is inconsistent with levels"
        levels = level_order
    elif hasattr(data.obs[feature_name], 'cat'):
        levels = data.obs[feature_name].cat.categories.to_list()
    else:
        levels = pd.unique(data.obs[feature_name])
    n_levels = len(levels)
    feature_totals = np.zeros([n_levels, data.X.shape[1]])

    for level in range(n_levels):
        l_indices = np.where(data.obs[feature_name] == levels[level])
        feature_totals[level] = np.sum(data.X[l_indices], axis=0)

    g = stackbar(
        feature_totals,
        type_names=type_names,
        title=feature_name,
        level_names=levels,
        figsize=figsize,
        dpi=dpi,
        cmap=cmap,
        plot_legend=plot_legend,
        )
       # Add labels and title
    plt.xlabel('')
    plt.ylabel('Proportion of Cell Types. Normal')
    plt.title('')
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()


In [ ]:
grouped = adata.obs.groupby(['cell_type', 'origin']).size().reset_index(name='cell_count')

In [ ]:
grouped

In [ ]:
# Create a Boolean mask for the cells that you want to keep
mask = ~((adata.obs['origin'] == 'normal_adjacent') & (adata.obs['cell_type'] == 'malignant cell'))

# Subset the adata object using the mask to exclude the specific cells
adata = adata[mask].copy()

# Now `adata_filtered` contains all cells except those with origin == 'normal_adjacent' and cell_type == 'malignant cell'


In [ ]:
grouped = adata.obs.groupby(['cell_type', 'origin']).size().reset_index(name='cell_count')
grouped

In [ ]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [ ]:
set(adata_normal.obs.cell_type)

In [ ]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [ ]:
set(adata_normal.obs.cell_type)

In [ ]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [ ]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [ ]:
cell_type_counts = adata.obs.groupby(['sex', 'cell_type']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(6, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Cell proportion')
plt.title('Compositional Analysis of Cell Types')
plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_cell_type.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata.obs.groupby(['sex', 'dataset']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Cell proportion')
plt.title('Compositional Analysis Sex')
plt.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_dataset_sex.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata.obs.groupby(['origin', 'dataset']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Cell proportion')
plt.title('Compositional Analysis Origin')
plt.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_origin_dataset.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata.obs.groupby(['origin', 'cell_type']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Cell proportion')
plt.title('Compositional Analysis Origin')
plt.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_origin_cell_type.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata.obs.groupby(['cell_type', 'sex']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)
ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 6))

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.1%}', ha='center', va='center')

# Remove the plot frame (all spines)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['left'].set_visible(False)
#ax.spines['bottom'].set_visible(False)

# Add labels and title
plt.xlabel('Cell Type')
plt.ylabel('Proportion of Cell Types')
plt.title('Compositional Analysis of Cell Types & Sex')
plt.legend(title='', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_cell_type_sex.png', bbox_inches='tight')
# Show the plot
plt.show()


In [ ]:
cell_type_counts

In [ ]:
cell_type_proportions

In [ ]:
data = adata.obs[['sex', 'cell_type', 'sample']]
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform wilcoxonTest for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)

# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Tumor & Normal. Wilcoxon test')

# Adjust layout
plt.tight_layout()

plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_tn.png', bbox_inches='tight')
# Show the plot
plt.show()


In [ ]:
data = adata.obs[['sex', 'cell_type', 'sample']]
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
vswarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform wilcoxonTest for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    #if p_value < significance_level:
    x_position = list(cell_types).index(cell_type)
    y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
    plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)

# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Tumor & Normal. Wilcoxon test.')

# Adjust layout
plt.tight_layout()

plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_tn_allpvalues.png', bbox_inches='tight')
# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata[adata.obs["origin"]=="tumor_primary"].obs.groupby(['sex', 'cell_type']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(6, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Proportion of Cell Types')
plt.title('Compositional Analysis of Cell Types. Tumor')
plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_cell_type_tumor.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
len(adata.obs)

In [ ]:
adata.obs.origin.value_counts()

In [ ]:
cell_type_counts

In [ ]:
cell_type_proportions

In [ ]:
data = adata_tumor.obs[['sex', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)

# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Tumor. Wilcoxon test.')

# Adjust layout
plt.tight_layout()

plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_t.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
data = adata_tumor.obs[['sex', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()

    # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
        
    #if p_value < significance_level:
    x_position = list(cell_types).index(cell_type)
    y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
    plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)

# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Tumor. Wilcoxon test.')

# Adjust layout
plt.tight_layout()

plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_t_allpvalues.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts = adata_normal.obs.groupby(['sex', 'cell_type']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(6, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Proportion of Cell Types')
plt.title('Compositional Analysis of Cell Types. Normal')
plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_cell_type_normal.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
cell_type_counts

In [ ]:
cell_type_proportions

In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_normal.obs[['sex', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)


# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Normal. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_n.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_normal.obs[['sex', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['sex', 'sample']).size()

cell_type_fractions = data.groupby(['sex', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['sex'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(12, 6))

# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, showfliers=False, hue_order=["male", "female"])

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='sex', data=cell_type_fractions, size=3, hue_order=["male", "female"], dodge=True)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'male')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['sex'] == 'female')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    #if p_value < significance_level:
    x_position = list(cell_types).index(cell_type)
    y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
    plt.text(x_position, y_max, f'p={p_value:.3f}', ha='center', va='bottom', color='black', fontsize=10)




# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="Sex", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Sex and sample. Normal. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_n_allpvalues.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
adata_female = adata[adata.obs["sex"]=="female"]

In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_female.obs[['origin', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['origin', 'sample']).size()

cell_type_fractions = data.groupby(['origin', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['origin'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(15, 6))
custom_palette = {'tumor_primary': '#D21826', 'normal_adjacent': '#2B83BA'}


# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, showfliers=False, hue_order=["tumor_primary", "normal_adjacent"],palette=custom_palette)

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, size=3, hue_order=["tumor_primary", "normal_adjacent"], dodge=True,palette=custom_palette)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'tumor_primary')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'normal_adjacent')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)


# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="origin", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Origin and sample. Female. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_female.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_female.obs[['origin', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['origin', 'sample']).size()

cell_type_fractions = data.groupby(['origin', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['origin'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(15, 6))
custom_palette = {'tumor_primary': '#D21826', 'normal_adjacent': '#2B83BA'}


# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, showfliers=False, hue_order=["tumor_primary", "normal_adjacent"],palette=custom_palette)

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, size=3, hue_order=["tumor_primary", "normal_adjacent"], dodge=True,palette=custom_palette)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'tumor_primary')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'normal_adjacent')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    #if p_value < significance_level:
    x_position = list(cell_types).index(cell_type)
    y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
    plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)


# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="origin", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. origin and sample. Normal. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


#plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_female.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
adata_male = adata[adata.obs["sex"]=="male"]

In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_male.obs[['origin', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['origin', 'sample']).size()

cell_type_fractions = data.groupby(['origin', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['origin'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(15, 6))
# Define the custom red and green palette
custom_palette = {'tumor_primary': '#D21826', 'normal_adjacent': '#2B83BA'}


# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, showfliers=False, hue_order=["tumor_primary", "normal_adjacent"],palette=custom_palette)

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, size=3, hue_order=["tumor_primary", "normal_adjacent"],palette=custom_palette, dodge=True)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'tumor_primary')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'normal_adjacent')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    if p_value < significance_level:
        x_position = list(cell_types).index(cell_type)
        y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
        plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)


# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="origin", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. Origin and sample. Male. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_male.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# Assuming 'sex', 'cell_type', and 'donor_id' are columns in your dataset (adata.obs in your case)
data = adata_female.obs[['origin', 'cell_type', 'sample']]

# Step 1: Calculate the cell type proportions for each sex and donor
total_cells_by_sex_donor = data.groupby(['origin', 'sample']).size()

cell_type_fractions = data.groupby(['origin', 'sample', 'cell_type']).size().reset_index(name='count')
cell_type_fractions['fraction'] = cell_type_fractions.apply(
    lambda row: row['count'] / total_cells_by_sex_donor[(row['origin'], row['sample'])], axis=1
)

# Step 2: Create the boxplot and swarmplot
plt.figure(figsize=(15, 6))
custom_palette = {'tumor_primary': '#D21826', 'normal_adjacent': '#2B83BA'}


# Create the boxplot
boxplot = sns.boxplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, showfliers=False, hue_order=["tumor_primary", "normal_adjacent"],palette=custom_palette)

# Overlay the swarmplot
swarmplot = sns.swarmplot(x='cell_type', y='fraction', hue='origin', data=cell_type_fractions, size=3, hue_order=["tumor_primary", "normal_adjacent"], dodge=True,palette=custom_palette)

# Step 3: Perform Mann-Whitney U Test for significance and display p-values
significance_level = 0.05  

cell_types = cell_type_fractions['cell_type'].unique()


for cell_type in cell_types:
    male_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'tumor_primary')]['fraction'].dropna()
    female_fractions = cell_type_fractions[(cell_type_fractions['cell_type'] == cell_type) & (cell_type_fractions['origin'] == 'normal_adjacent')]['fraction'].dropna()
          # Ensure the lengths are equal by truncating the longer list
    min_length = min(len(male_fractions), len(female_fractions))
    male_fractions = male_fractions[:min_length]
    female_fractions = female_fractions[:min_length]
    
    # Perform wilcoxon Test
    stat, p_value = wilcoxon(male_fractions, female_fractions)
    #if p_value < significance_level:
    x_position = list(cell_types).index(cell_type)
    y_max = cell_type_fractions[cell_type_fractions['cell_type'] == cell_type]['fraction'].max()
    plt.text(x_position, y_max, f'p={p_value:.3e}', ha='center', va='bottom', color='black', fontsize=10)


# Step 4: Remove duplicate legends
handles, labels = boxplot.get_legend_handles_labels()
plt.legend().remove()

# Add a single legend using the collected handles and labels
plt.legend(handles=handles[:2], labels=labels[:2], title="origin", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.xlabel('')
plt.ylabel('Cell fraction')
plt.title('Cell Type Fractions. origin and sample. Male. Wilcoxon test.')

# Adjust layout
plt.tight_layout()


#plt.savefig(f'{resDir}/stacked_barplot_swarm_willcoxon_male.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
adata_cd4 = adata[adata.obs["cell_type"]=="CD4+ T cell"]

In [ ]:
cell_type_counts = adata_cd4.obs.groupby(['sex', 'origin']).size().unstack(fill_value=0)
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

ax = cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 8))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Annotating percentages
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2, y + height / 2, f'{height:.2%}', ha='center', va='center')


# Add labels and title
plt.xlabel('')
plt.ylabel('Cell proportion')
plt.title('Compositional Analysis Origin')
plt.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()


# Save the plot
plt.savefig(f'{resDir}/stacked_barplot_origin_cell_type.png', bbox_inches='tight')

# Show the plot
plt.show()
